In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [124]:
def load_and_preprocess_data(filepath):
    act = pd.read_csv("../data/dailyActivity_merged_fin_sum.csv")
    
    act['ActivityDate'] = pd.to_datetime(act['ActivityDate'])
    
    # Mile -> Km 변환
    distance_cols = [col for col in act.columns if 'Distance' in col]
    act[distance_cols] = (act[distance_cols] * 1.60934).round(2)
        
    act['Id'] = act['Id'].astype(str)
    
    return act

act = load_and_preprocess_data("../data/dailyActivity_merged_fin_sum.csv")


In [125]:
def remove_non_wear_days(df):
    non_wear = (df['TotalSteps'] == 0) & \
               (df['TotalDistance'] == 0) & \
               (df['SedentaryMinutes'] >= 1380)
    
    print(f"미착용일 제거: {non_wear.sum()}개")
    return df[~non_wear].copy()

act = remove_non_wear_days(act)

미착용일 제거: 124개


In [126]:
def create_derived_features(df):
    df = df.copy()
    
    df['weekday'] = df['ActivityDate'].dt.day_name()
    df['is_weekend'] = df['weekday'].isin(['Saturday', 'Sunday'])
    
    df['TotalActiveMinutes'] = (
        df['VeryActiveMinutes'] + 
        df['FairlyActiveMinutes'] + 
        df['LightlyActiveMinutes']
    )
    
    df['SedentaryRatio'] = df['SedentaryMinutes'] / 1440
    
    df['Intensity_Score'] = (
        (df['VeryActiveMinutes'] * 2) + 
        (df['FairlyActiveMinutes'] * 1.5) + 
        (df['LightlyActiveMinutes'] * 1)
    )
    
    df['Efficiency'] = np.where(
        df['TotalActiveMinutes'] > 0,
        df['Intensity_Score'] / df['TotalActiveMinutes'],
        0
    )
    
    df['CaloriesPerKm'] = np.where(
        df['TotalDistance'] > 0,
        df['Calories'] / df['TotalDistance'],
        np.nan
    )
    
    return df

act = create_derived_features(act)

In [127]:
def classify_day_type(df):
    df = df.copy()
    
    conditions = [
        (df['TotalSteps'] >= 7000) | (df['TotalActiveMinutes'] >= 60),
        (df['SedentaryRatio'] >= 0.75) & (df['TotalSteps'] > 0),
        (df['TotalSteps'] < 3000) & (df['SedentaryRatio'] >= 0.50) & (df['TotalSteps'] > 0)
    ]
    
    choices = ['Active Day', 'Over-Sedentary Day', 'Low Engagement Day']
    df['DayType'] = np.select(conditions, choices, default='Normal Day')
    
    return df

act = classify_day_type(act)


In [128]:
def create_calorie_groups(df):
    df = df.copy()
    
    bins = [0, 1500, 2000, 2500, float('inf')]
    labels = ['1000-1500', '1500-2000', '2000-2500', '2500+']
    df['CalorieGroup'] = pd.cut(df['Calories'], bins=bins, labels=labels, right=False)
    
    return df

act = create_calorie_groups(act)

In [129]:
def calc_max_streak(dates):
    s = pd.to_datetime(dates, errors='coerce')
    uniq = pd.Series(s.dropna().dt.normalize().unique())
    
    if uniq.empty:
        return 0
    
    uniq = uniq.sort_values().to_list()
    max_streak = 1
    current = 1
    
    for i in range(1, len(uniq)):
        if (uniq[i] - uniq[i-1]).days == 1:
            current += 1
            max_streak = max(max_streak, current)
        else:
            current = 1
    
    return max_streak

streak_df = act.groupby("Id")["ActivityDate"].apply(calc_max_streak).reset_index(name="streak_days")
act = act.merge(streak_df, on="Id", how="left")
act.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,...,weekday,is_weekend,TotalActiveMinutes,SedentaryRatio,Intensity_Score,Efficiency,CaloriesPerKm,DayType,CalorieGroup,streak_days
0,1503960366,2016-03-25,11004,11.44,11.44,0.0,4.14,0.74,6.55,0.0,...,Friday,False,250,0.558333,289.0,1.156000,159.003497,Active Day,1500-2000,48
1,1503960366,2016-03-26,17609,18.59,18.59,0.0,11.14,1.17,6.29,0.0,...,Saturday,True,380,0.408333,477.5,1.256579,115.868747,Active Day,2000-2500,48
2,1503960366,2016-03-27,12736,13.73,13.73,0.0,7.50,0.26,5.97,0.0,...,Sunday,True,329,0.420139,387.5,1.177812,141.587764,Active Day,1500-2000,48
3,1503960366,2016-03-28,13231,14.37,14.37,0.0,5.13,1.27,7.97,0.0,...,Monday,False,283,0.750000,332.0,1.173145,134.446764,Active Day,1500-2000,48
4,1503960366,2016-03-29,12041,12.63,12.63,0.0,3.48,1.75,7.42,0.0,...,Tuesday,False,299,0.529861,341.0,1.140468,149.326999,Active Day,1500-2000,48


In [130]:
def remove_outliers_iqr(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame: # IQR 방법으로 이상치 제거
  
    df_clean = df.copy()
    
    print("\n=== IQR 이상치 제거 ===")
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        before_count = len(df_clean)
        df_clean = df_clean[
            (df_clean[col] >= lower_bound) & 
            (df_clean[col] <= upper_bound)
        ]
        after_count = len(df_clean)
        
        if before_count != after_count:
            print(f"{col}: {before_count - after_count}개 제거")
    
    return df_clean
outlier_cols = ['TotalSteps', 'TotalDistance', 'TrackerDistance',
                'LoggedActivitiesDistance', 'VeryActiveDistance',
                'ModeratelyActiveDistance', 'LightActiveDistance',
                'VeryActiveMinutes','FairlyActiveMinutes', 
                'LightlyActiveMinutes', 
                'SedentaryMinutes', 'Calories','CaloriesPerKm']

act = remove_outliers_iqr(act, outlier_cols)


=== IQR 이상치 제거 ===
TotalSteps: 19개 제거
TotalDistance: 16개 제거
TrackerDistance: 1개 제거
LoggedActivitiesDistance: 53개 제거
VeryActiveDistance: 78개 제거
ModeratelyActiveDistance: 75개 제거
LightActiveDistance: 6개 제거
VeryActiveMinutes: 59개 제거
FairlyActiveMinutes: 34개 제거
LightlyActiveMinutes: 3개 제거
SedentaryMinutes: 2개 제거
Calories: 13개 제거
CaloriesPerKm: 98개 제거


In [131]:
def filter_date_range(df: pd.DataFrame, 
                      start_date: Optional[str] = "2016-04-01",
                      end_date: Optional[str] = None) -> pd.DataFrame:
    """날짜 범위 필터링 (2016-04-01부터)"""
    start_date = pd.to_datetime(start_date)
        
    if end_date is None:
        end_date = df['ActivityDate'].max()
    else:
        end_date = pd.to_datetime(end_date)
    
    filtered = df[
        (df['ActivityDate'] >= start_date) & 
        (df['ActivityDate'] <= end_date)
    ].copy()
    
    print(f"✓ 날짜 필터링: {start_date.date()} ~ {end_date.date()}")
    print(f"  데이터 행 수: {len(filtered)}")
    
    return filtered

act = filter_date_range(act)


✓ 날짜 필터링: 2016-04-01 ~ 2016-05-12
  데이터 행 수: 749


In [132]:
act.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,...,weekday,is_weekend,TotalActiveMinutes,SedentaryRatio,Intensity_Score,Efficiency,CaloriesPerKm,DayType,CalorieGroup,streak_days
7,1503960366,2016-04-01,12262,12.67,12.67,0.0,5.34,1.34,5.86,0.0,...,Friday,False,268,0.601389,325.5,1.214552,147.434886,Active Day,1500-2000,48
8,1503960366,2016-04-10,10057,11.23,11.23,0.0,6.44,0.79,3.99,0.0,...,Sunday,True,225,0.511806,275.5,1.224444,156.277827,Active Day,1500-2000,48
9,1503960366,2016-04-11,10990,11.68,11.68,0.0,3.28,0.92,7.48,0.0,...,Monday,False,256,0.593750,289.0,1.128906,155.051370,Active Day,1500-2000,48
10,1503960366,2016-04-12,13386,13.90,13.90,0.0,3.03,0.89,9.96,0.0,...,Tuesday,False,375,0.527778,406.5,1.084000,146.402878,Active Day,2000-2500,48
11,1503960366,2016-04-13,10735,11.22,11.22,0.0,2.53,1.11,7.58,0.0,...,Wednesday,False,257,0.538889,287.5,1.118677,160.160428,Active Day,1500-2000,48


In [133]:
act.dtypes

Id                                     str
ActivityDate                datetime64[us]
TotalSteps                           int64
TotalDistance                      float64
TrackerDistance                    float64
LoggedActivitiesDistance           float64
VeryActiveDistance                 float64
ModeratelyActiveDistance           float64
LightActiveDistance                float64
SedentaryActiveDistance            float64
VeryActiveMinutes                    int64
FairlyActiveMinutes                  int64
LightlyActiveMinutes                 int64
SedentaryMinutes                     int64
Calories                             int64
weekday                                str
is_weekend                            bool
TotalActiveMinutes                   int64
SedentaryRatio                     float64
Intensity_Score                    float64
Efficiency                         float64
CaloriesPerKm                      float64
DayType                                str
CalorieGrou

In [134]:
ACTIVITY_LEVELS = {
    'low': (0, 1500),
    'moderate': (1500, 2000),
    'active': (2000, 2500),
    'high': (2500, 999999)
}

PERSONA_TYPES = {
    'newbie': {'name': '입문자형', 'level': 'low', 'desc': '막 시작한 사람'},
    'beginner': {'name': '초보자형', 'level': 'moderate', 'stable': False, 'desc': '운동 수준 낮고 습관화 안됨'},
    'turtle': {'name': '거북이형', 'level': 'moderate', 'stable': True, 'desc': '습관은 있는데 강도 낮음'},
    'burst': {'name': '벼락치기형', 'level': 'active', 'stable': False, 'desc': '운동은 잘하는데 자주 안함'},
    'ideal': {'name': '모범생형', 'level': 'active', 'stable': True, 'desc': '이상적 타입'},
    'lazy_genius': {'name': '게으른 천재형', 'level': 'high', 'stable': False, 'desc': '수준은 높은데 횟수 적음'},
    'veteran': {'name': '고인물형', 'level': 'high', 'stable': True, 'desc': '고수. 부상주의'}
}

In [135]:

class PersonaClassifier:
    def __init__(self):
        self.avg_active_days = 32
    
    def get_user_metrics(self, df, user_id):
        user_data = df[df['Id'] == user_id]
        
        metrics = {
            'user_id': user_id,
            'avg_calories': user_data['Calories'].mean(),
            'avg_steps': user_data['TotalSteps'].mean(),
            'total_calories': user_data['Calories'].sum(),
            'total_steps': user_data['TotalSteps'].sum(),
            'total_distance': user_data['TotalDistance'].sum(),
            'active_days': len(user_data),
            'max_streak': user_data['streak_days'].iloc[0] if len(user_data) > 0 else 0,
            'avg_efficiency': user_data['Efficiency'].mean(),
            'over_sedentary_ratio': (user_data['DayType'] == 'Over-Sedentary Day').sum() / len(user_data)
        }
        
        return metrics
    
    def get_activity_level(self, avg_calories):
        for level, (min_cal, max_cal) in ACTIVITY_LEVELS.items():
            if min_cal <= avg_calories < max_cal:
                return level
        return 'low'
    
    def is_stable(self, active_days):
        return active_days >= self.avg_active_days
    
    def classify_persona(self, metrics):
        level = self.get_activity_level(metrics['avg_calories'])
        stable = self.is_stable(metrics['active_days'])
        
        if level == 'low':
            return 'newbie'
        elif level == 'moderate':
            return 'turtle' if stable else 'beginner'
        elif level == 'active':
            return 'ideal' if stable else 'burst'
        else:
            return 'veteran' if stable else 'lazy_genius'
    
    def get_goals(self, persona_type):
        goals = {
            'newbie': {
                'short': '하루 3,000보 달성 주 3회',
                'medium': '1500 kcal 그룹 진입',
                'long': 'Beginner로 성장',
                'programs': ['3분 스트레칭 챌린지', '출퇴근길 걷기', '기초 운동 루틴']
            },
            'beginner': {
                'short': '주 3회 운동하기',
                'medium': '일 평균 5,000보 달성',
                'long': 'Turtle 또는 Burst로 성장',
                'programs': ['걷기 챌린지', '홈트 시작반', '운동 알림']
            },
            'turtle': {
                'short': '고강도 운동 추가 (주 1회)',
                'medium': '2000 kcal 그룹 진입',
                'long': 'Ideal로 성장',
                'programs': ['인터벌 트레이닝', '계단 오르기', '운동 강도 높이기']
            },
            'burst': {
                'short': '주 5회 운동 도전',
                'medium': '운동 일정 정기화',
                'long': 'Ideal로 성장',
                'programs': ['운동 습관 챌린지', '리마인더 설정', '커뮤니티 참여']
            },
            'ideal': {
                'short': '현재 수준 유지',
                'medium': '운동 다양화',
                'long': 'Veteran 도전 or 현상유지',
                'programs': ['크로스핏', '요가 병행', '새로운 운동']
            },
            'lazy_genius': {
                'short': '운동 빈도 늘리기 (주 4회→5회)',
                'medium': '연속 7일 달성',
                'long': 'Veteran으로 성장',
                'programs': ['운동 스케줄링', '알림 강화', '습관화 프로그램']
            },
            'veteran': {
                'short': '부상 예방 관리',
                'medium': '휴식 및 회복 최적화',
                'long': '운동 다양화 및 전문화',
                'programs': ['스마트 회복 프로그램', '부상 방지 스트레칭', '전문 코칭']
            }
        }
        return goals.get(persona_type, goals['newbie'])
    
    def classify_user(self, df, user_id):
        metrics = self.get_user_metrics(df, user_id)
        persona_key = self.classify_persona(metrics)
        persona_info = PERSONA_TYPES[persona_key]
        goals = self.get_goals(persona_key)
        
        return {
            'user_id': user_id,
            'persona': persona_info['name'],
            'level': persona_info['level'],
            'desc': persona_info['desc'],
            'metrics': metrics,
            **goals
        }
    
    def generate_report(self, result):
        m = result['metrics']
        
        report = f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  BellaBuddy 월간 분석 리포트
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

페르소나: {result['persona']}
레벨: {result['level']}
{result['desc']}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
이번 달 활동
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

총 걸음수: {m['total_steps']:,} 보
총 거리: {m['total_distance']:.1f} km
총 칼로리: {m['total_calories']:,} kcal

활동일수: {m['active_days']}일
최대 연속: {m['max_streak']}일

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
다음 목표
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1주 목표: {result['short']}
1개월 목표: {result['medium']}
3개월 목표: {result['long']}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
추천 프로그램
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""
        for i, prog in enumerate(result['programs'], 1):
            report += f"{i}. {prog}\n"
        
        report += "━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n"
        
        return report

In [136]:

class PersonaInfo: #페르소나 분류 결과
    
    user_id: str
    persona: PersonaType
    activity_level: ActivityLevel
    persistence_type: PersistenceType
    
    # 상세 지표
    metrics: UserMetrics
    
    # 목표 및 추천
    short_term_goal: str = ""
    medium_term_goal: str = ""
    long_term_goal: str = ""
    recommended_programs: List[str] = field(default_factory=list)
    
    def get_summary(self) -> str: #페르소나 요약 정보
        
        return f"""
        {'='*50}
        사용자 ID: {self.user_id}
        페르소나: {self.persona.name_kr}
        활동 등급: {self.activity_level.value}
        지속성: {self.persistence_type.value}

        [주요 지표]
        - 평균 칼로리: {self.metrics.avg_calories:.0f} kcal
        - 평균 걸음수: {self.metrics.avg_steps:.0f} 보
        - 활동 일수: {self.metrics.active_days}/{self.metrics.total_days}일
        - 효율성: {self.metrics.avg_efficiency:.2f}

        [특징]
        {self.persona.description}

        [단기 목표 (1주)]
        {self.short_term_goal}

        [중기 목표 (1개월)]
        {self.medium_term_goal}

        [장기 목표 (3개월)]
        {self.long_term_goal}

        [추천 프로그램]
        {"\n".join(f'• {prog}' for prog in self.recommended_programs)}
        {'='*50}
        """


In [137]:
class PersonaClassifier:
    def __init__(self):
        self.avg_active_days = 32  # 전체 평균
    
    def get_user_metrics(self, df, user_id):
        user_data = df[df['Id'] == user_id]
        
        metrics = {
            'user_id': user_id,
            'avg_calories': user_data['Calories'].mean(),
            'avg_steps': user_data['TotalSteps'].mean(),
            'total_calories': user_data['Calories'].sum(),
            'total_steps': user_data['TotalSteps'].sum(),
            'total_distance': user_data['TotalDistance'].sum(),
            'active_days': len(user_data),
            'max_streak': user_data['streak_days'].iloc[0] if len(user_data) > 0 else 0,
            'avg_efficiency': user_data['Efficiency'].mean(),
            'over_sedentary_ratio': (user_data['DayType'] == 'Over-Sedentary Day').sum() / len(user_data)
        }
        
        return metrics
    
    def get_activity_level(self, avg_calories):
        for level, (min_cal, max_cal) in ACTIVITY_LEVELS.items():
            if min_cal <= avg_calories < max_cal:
                return level
        return 'low'
    
    def is_stable(self, active_days):
        return active_days >= self.avg_active_days
    
    def classify_persona(self, metrics):
        level = self.get_activity_level(metrics['avg_calories'])
        stable = self.is_stable(metrics['active_days'])
        
        if level == 'low':
            return 'newbie'
        elif level == 'moderate':
            return 'turtle' if stable else 'beginner'
        elif level == 'active':
            return 'ideal' if stable else 'burst'
        else:  # high
            return 'veteran' if stable else 'lazy_genius'
    
    def get_goals(self, persona_type):
        goals = {
            'newbie': {
                'short': '하루 3,000보 달성 주 3회',
                'medium': '1500 kcal 그룹 진입',
                'long': 'Beginner로 성장',
                'programs': ['3분 스트레칭 챌린지', '출퇴근길 걷기', '기초 운동 루틴']
            },
            'beginner': {
                'short': '주 3회 운동하기',
                'medium': '일 평균 5,000보 달성',
                'long': 'Turtle 또는 Burst로 성장',
                'programs': ['걷기 챌린지', '홈트 시작반', '운동 알림']
            },
            'turtle': {
                'short': '고강도 운동 추가 (주 1회)',
                'medium': '2000 kcal 그룹 진입',
                'long': 'Ideal로 성장',
                'programs': ['인터벌 트레이닝', '계단 오르기', '운동 강도 높이기']
            },
            'burst': {
                'short': '주 5회 운동 도전',
                'medium': '운동 일정 정기화',
                'long': 'Ideal로 성장',
                'programs': ['운동 습관 챌린지', '리마인더 설정', '커뮤니티 참여']
            },
            'ideal': {
                'short': '현재 수준 유지',
                'medium': '운동 다양화',
                'long': 'Veteran 도전 or 현상유지',
                'programs': ['크로스핏', '요가 병행', '새로운 운동']
            },
            'lazy_genius': {
                'short': '운동 빈도 늘리기 (주 4회→5회)',
                'medium': '연속 7일 달성',
                'long': 'Veteran으로 성장',
                'programs': ['운동 스케줄링', '알림 강화', '습관화 프로그램']
            },
            'veteran': {
                'short': '부상 예방 관리',
                'medium': '휴식 및 회복 최적화',
                'long': '운동 다양화 및 전문화',
                'programs': ['스마트 회복 프로그램', '부상 방지 스트레칭', '전문 코칭']
            }
        }
        return goals.get(persona_type, goals['newbie'])
    
    def classify_user(self, df, user_id):
        metrics = self.get_user_metrics(df, user_id)
        persona_key = self.classify_persona(metrics)
        persona_info = PERSONA_TYPES[persona_key]
        goals = self.get_goals(persona_key)
        
        return {
            'user_id': user_id,
            'persona': persona_info['name'],
            'level': persona_info['level'],
            'desc': persona_info['desc'],
            'metrics': metrics,
            **goals
        }
    
    def generate_report(self, result):
        m = result['metrics']
        
        report = f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  BellaBuddy 월간 분석 리포트
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

페르소나: {result['persona']}
레벨: {result['level']}
{result['desc']}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
이번 달 활동
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

총 걸음수: {m['total_steps']:,} 보
총 거리: {m['total_distance']:.1f} km
총 칼로리: {m['total_calories']:,} kcal

활동일수: {m['active_days']}일
최대 연속: {m['max_streak']}일

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
다음 목표
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1주 목표: {result['short']}
1개월 목표: {result['medium']}
3개월 목표: {result['long']}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
추천 프로그램
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""
        for i, prog in enumerate(result['programs'], 1):
            report += f"{i}. {prog}\n"
        
        report += "━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n"
        
        return report


# 사용법
def analyze_user(df, user_id):
    classifier = PersonaClassifier()
    result = classifier.classify_user(df, user_id)
    print(classifier.generate_report(result))


# 테스트
analyze_user(act, "1624580081")

# 전체 사용자 ID 리스트
user_ids = act['Id'].unique().tolist()
print(user_ids)



━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  BellaBuddy 월간 분석 리포트
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

페르소나: 입문자형
레벨: low
막 시작한 사람

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
이번 달 활동
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

총 걸음수: 163,022 보
총 거리: 172.1 km
총 칼로리: 50,315 kcal

활동일수: 35일
최대 연속: 49일

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
다음 목표
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1주 목표: 하루 3,000보 달성 주 3회
1개월 목표: 1500 kcal 그룹 진입
3개월 목표: Beginner로 성장

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
추천 프로그램
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. 3분 스트레칭 챌린지
2. 출퇴근길 걷기
3. 기초 운동 루틴
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

['1503960366', '1624580081', '1644430081', '1844505072', '1927972279', '2022484408', '2026352035', '2320127002', '2347167796', '2873212765', '3372868164', '3977333714', '4020332650', '4057192912', '4319703577', '4388161847', '4445114986', '4558609924', '4702921684', '5553957443', '5577150313', '6117666160', '6290855005', '6775888955', '6962181067', '70077441

In [138]:
def analyze_single_user(df: pd.DataFrame, user_id: str):
    """사용자 ID 입력 → 자동으로 페르소나 분석 + 상세 리포트 출력"""
    classifier = PersonaClassifier()
    persona_info = classifier.classify_user(df, user_id)
    
    # 리포트 생성 및 출력
    report = classifier.generate_report(persona_info)
    print(report)

In [139]:
# 원하는 사용자 ID 입력
analyze_single_user(act, "1624580081")


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  BellaBuddy 월간 분석 리포트
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

페르소나: 입문자형
레벨: low
막 시작한 사람

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
이번 달 활동
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

총 걸음수: 163,022 보
총 거리: 172.1 km
총 칼로리: 50,315 kcal

활동일수: 35일
최대 연속: 49일

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
다음 목표
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1주 목표: 하루 3,000보 달성 주 3회
1개월 목표: 1500 kcal 그룹 진입
3개월 목표: Beginner로 성장

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
추천 프로그램
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. 3분 스트레칭 챌린지
2. 출퇴근길 걷기
3. 기초 운동 루틴
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━



In [140]:
# 사용자 ID 리스트만
user_ids = act['Id'].unique().tolist()
print(user_ids)

['1503960366', '1624580081', '1644430081', '1844505072', '1927972279', '2022484408', '2026352035', '2320127002', '2347167796', '2873212765', '3372868164', '3977333714', '4020332650', '4057192912', '4319703577', '4388161847', '4445114986', '4558609924', '4702921684', '5553957443', '5577150313', '6117666160', '6290855005', '6775888955', '6962181067', '7007744171', '7086361926', '8053475328', '8253242879', '8378563200', '8583815059', '8792009665', '8877689391']
